In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

# SELLER ANALYSIS

Computing the P&L of each seller where we need the
* Revenue
* Cost
* Profit

Where theres
* a 10% fee per sale
* 80 BRL subscription fee per month
* a review cost of ```{1: 100, 2: 50, 3: 40, 4: 0, 5: 0}``` as key and cost in BRL as value

In [2]:
def get_review_score(self):
    """
    Returns a DataFrame with:
    'seller_id', 'share_of_five_stars', 'share_of_one_stars', 'review_score', 'cost_of_reviews'
    """

    orders_reviews = self.order.get_review_score()
    orders_sellers = self.data['order_items'][['order_id', 'seller_id'
                                               ]].drop_duplicates()

    df = orders_sellers.merge(orders_reviews, on='order_id')

    df['cost_of_review'] = df.review_score.map({
        1: 100,
        2: 50,
        3: 40,
        4: 0,
        5: 0
    })

    df_grouped_by_sellers = df.groupby('seller_id', as_index=False).agg({
        'dim_is_one_star':
        'mean',
        'dim_is_five_star':
        'mean',
        'review_score':
        'mean',
        'cost_of_review':
        'sum'
    })
    df_grouped_by_sellers.columns = [
        'seller_id', 'share_of_one_stars', 'share_of_five_stars',
        'review_score', 'cost_of_reviews'
    ]

    return df_grouped_by_sellers

In [3]:
def get_training_data(self):
    """
    Returns a DataFrame with:
    ['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
    'wait_time', 'date_first_sale', 'date_last_sale', 'months_on_olist',
    'share_of_one_stars', 'share_of_five_stars', 'review_score',
    'cost_of_reviews', 'n_orders', 'quantity', 'quantity_per_order',
    'sales', 'revenues', 'profits']
    """
    training_set =\
        self.get_seller_features()\
            .merge(
            self.get_seller_delay_wait_time(), on='seller_id'
        ).merge(
            self.get_active_dates(), on='seller_id'
        ).merge(
            self.get_review_score(), on='seller_id'
        ).merge(
            self.get_quantity(), on='seller_id'
        ).merge(
            self.get_sales(), on='seller_id'
        )

    # Add seller economics (revenues, profits)
    olist_monthly_fee = 80
    olist_sales_cut = 0.1

    training_set['revenues'] = training_set['months_on_olist'] * olist_monthly_fee\
        + olist_sales_cut * training_set['sales']

    training_set['profits'] = training_set['revenues'] - training_set[
        'cost_of_reviews']

    return training_set

In [4]:
from olist.seller import Seller as SellerUpdated
sellers = SellerUpdated().get_training_data()

In [5]:
sellers.head(2)

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,n_orders,quantity,quantity_per_order,sales,share_of_one_stars,share_of_five_stars,review_score,cost_of_reviews
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.0,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,3,3,1.000,218.70,0.333333,0.333333,3.00,140
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.0,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,40,41,1.025,11703.07,0.050000,0.725000,4.55,240


In [5]:
sorted_sellers = sellers.sort_values(by='profits')[['profits', 'n_orders', 'revenues']].reset_index()
sorted_sellers.head()

plt.figure(figsize=(10,5))
sorted_sellers.profits.plot()
plt.title('gross profit per seller')
plt.xlabel('seller id')

KeyError: 'profits'

NameError: name 'total_profits' is not defined